In [1]:
#### import tatqa_utils
import pandas as pd
import table_convert
import pyreadstat
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

import utils
from langchain_community.vectorstores import FAISS
from langchain_core.documents import Document
import numpy as np
from progress.bar import Bar
%load_ext autoreload
%autoreload 2


In [2]:
import os
with open('dataset_raw/openai.api.key', 'r') as filek: 
    openai_key = filek.read()
os.environ["OPENAI_API_KEY"] =  openai_key 

In [3]:
from langchain.globals import set_llm_cache
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAI
from langchain_community.cache import SQLiteCache

#llm = ChatOpenAI(temperature=0)
#llm = OpenAI(temperature=0) 
#set_llm_cache(SQLiteCache(database_path=".langchain.db"))

In [4]:
devdf = pd.read_json('dataset_raw/tatqa_dataset_dev.json')

In [130]:
def get_question(qid):
    for i, item in devdf.iterrows():
        for q in item['questions']:        
            if q['uid'] == qid:
                #table = item['table']['table']
                return (item['table'], q)
    return (None, None)

In [140]:
def gen_code3(llm, messages, question, value_list): 
    
    prompt = ChatPromptTemplate.from_messages(messages)

    output_parser = StrOutputParser()

    chain = prompt | llm | output_parser
    
    response = chain.invoke({"value_list": value_list, "question":question})
    code =  response.replace('```python','').replace('```','')
    
    return ("", code)

def exec_code(code, value_list):  
        try: 
            loc = locals()   
            if not "run()" in code:
                exec(code + f"\nr = run({value_list})\n", globals(), loc)
            else: 
                exec(code + "\nr = run()\n", globals(), loc)
            return loc['r']
        except Exception as e:
                s = '[Error]'+ str(e)
                print(s)
                return (s,'')
            

In [141]:
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

#llm = ChatOpenAI(model="gpt-3.5-turbo", temperature = 0, top_p = 1.0, seed=11)
llm = ChatOpenAI(model="gpt-3.5-turbo", 
temperature=0,
    #max_tokens=150,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0,
    seed=123)

In [252]:
 messages=[
         ("system","You are a helpful assistant with a subtask in a question-answering pipeline. The questions are related to a financial report. " + 
             "Financial report is stored as a list of annotated values. You will receive the financial report as an annotated value list and the question. "+
             "Your task is to generate a Python function that can calculate a numeric value that is the answer for the received question. "                            
         ),
        (
          "human",
          "Here is the financial report as a list of annotated values: {value_list}"
        ),         
        (
          "ai",
          "Ok, I received the value list."
        ),
        (
          "human",
          "Here is the question requires calculation on the financial report: {question}"
        ),
        (
          "ai",
          "Ok, I received the question."
        ),
        (
          "human",
          "Generate a Python function 'run(value_list)' that can answer the question using the list of annotated values! "+
            "The function must return a tuple (number, scale). The resulting number is a float with accuracy to two decimal places. Scale usually is thousand, million, billion, percent or an empty string. "+
            "The calculation usually involves two steps: a selection and a calculation on selected data. "+
            #"In the value list, each value represents one year. "+
            "If the question is about calculating the average value in a given year, you must calculate the average between the given year and the previous one. It is aka Average Book Value. ex. 2012_average = (2012_value + 2011_value)/2  "+
            "If the question is about calculating the change in average values in years, you must calculate the year average values (average between the given year and the previous one)  before difference calculation. ex. change_in _average_2011_2012 = (2012_value + 2011_value)/2 - (2011_value + 2010_value)/2 "
            "The code can be specific to the provided value list. " +
            "Do not generate explanation, nor example code, just the function."
        ),
        (
          "ai",
          "Ok, I have all the information. The Python function is as follows:"
        ),
      ]     


In [255]:
devdf = pd.read_json('dataset_raw/tatqa_dataset_dev.json')

#qid = 'ba6783f3-8207-419a-b407-3f688682caef'
#qid = 'a0414f81-8dc2-44b2-a441-2c9d9c805c4d'

#qid = 'bf7abd62-d9cd-48d2-8826-1457684019a3'
#qid = '4d259081-6da6-44bd-8830-e4de0031744c'
#qid = 'dc5e217a-a7b3-4fc9-ac0f-13d328f26b20'

#qid = '7cd3aedf-1291-4fea-bc9d-a25c65727b7b'
qid = '22e20f25-669a-46b9-8779-2768ba391955'
#qid = '65ec782c-691e-45df-b541-caecb85154ff'
#qid = 'a983501d-2eec-486d-9661-e520c7c8af5e'


_table, _q =  get_question(qid)
table = _table['table']

#[print(r) for r in table]

q = _q['question']
print(q)
values = table_convert.convert_table(table)
#response = chain.invoke({"value_list": values, "question":q})
#code =  response.replace('```python','').replace('```','')
p, code = gen_code3(llm, messages, q, values)
print(code)
(v, s) = exec_code(code, values)

if v == _q['answer']:
    print("\033[92m Success: " + str(v)+'\033[0m')
else:    
    print("\033[91m failure: " + str(v), 'good answer: ', _q['answer'],'\033[0m' )  

What is the change between 2018 and 2019 average free cash flow?

def run(value_list):
    free_cash_flow_2019 = 4411.0
    free_cash_flow_2018 = 4044.0
    average_free_cash_flow_2019 = (free_cash_flow_2019 + free_cash_flow_2018) / 2
    free_cash_flow_2017 = 3316.0
    average_free_cash_flow_2018 = (free_cash_flow_2018 + free_cash_flow_2017) / 2
    change_average_free_cash_flow_2018_2019 = average_free_cash_flow_2019 - average_free_cash_flow_2018
    return (round(change_average_free_cash_flow_2018_2019, 2), 'million')

 Success: 547.5


In [249]:
devdf = pd.read_json('dataset_raw/tatqa_dataset_dev.json')

qids = [
'ba6783f3-8207-419a-b407-3f688682caef',
'a0414f81-8dc2-44b2-a441-2c9d9c805c4d',
'bf7abd62-d9cd-48d2-8826-1457684019a3',
'4d259081-6da6-44bd-8830-e4de0031744c',
'dc5e217a-a7b3-4fc9-ac0f-13d328f26b20',

'7cd3aedf-1291-4fea-bc9d-a25c65727b7b',
'22e20f25-669a-46b9-8779-2768ba391955',
'65ec782c-691e-45df-b541-caecb85154ff',
'a983501d-2eec-486d-9661-e520c7c8af5e'
]
def get_answer(llm, messages, table, q):
    values = table_convert.convert_table(table)    
    p, code = gen_code3(llm, messages, q, values)    
    #print(code)
    (v, s) = exec_code(code, values)
    return  (v, s)

    

for qid in qids:
    _table, _q =  get_question(qid)
    table = _table['table']
    
    #[print(r) for r in table]
    
    q = _q['question']
    print(q, end='')
    
    r = []
    for i in [0,1,2]:        
       r.append(get_answer(llm, messages, table, q))
    
    if r[0][0] == r[1][0]:
        v = r[0][0]
    elif r[1][0] == r[2][0]:
        v = r[1][0]
    elif r[0][0] == r[2][0]:
        v = r[0][0] 
    else:
        v = None
        
    #(v, s) = get_answer(llm, messages, table, q)    
    
    if v == _q['answer']:
        print("\033[92m Success: " + str(v)+'\033[0m')
    else:    
        print("\033[91m failure: " + str(v), 'good answer: ', _q['answer'],'\033[0m' ) 

What is the ratio of IMFT’s total assets to total liabilities in 2019? failure: 1.0 good answer:  2.93 
What is the 2019 average defined contribution schemes? Success: 172.0
What is the 2019 average defined benefit schemes? Success: 50.5
What is the difference between 2019 average defined contribution schemes and 2019 average defined benefit schemes? failure: 138.33 good answer:  121.5 
What is the 2019 average free cash flow? failure: 4411.0 good answer:  4227.5 
What is the 2018 average free cash flow? failure: 3923.67 good answer:  3680 
What is the change between 2018 and 2019 average free cash flow?[Error]name 'free_cash_flow_2017' is not defined
 Success: 547.5
What was the change in Capital redemption reserve in 2019 from 2018? failure: 0.7 good answer:  0.2 
What was the average difference between EBITDA and underlying EBITDA for both FYs? failure: -2349.0 good answer:  3728 
